In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import json
from sklearn.metrics import mean_squared_error
from math import sqrt
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import time
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from sklearn.preprocessing import MinMaxScaler

In [2]:
pd.options.mode.copy_on_write = True 

In [3]:
steam_games_df_base = pd.read_json('steam_games_df.json', lines=True)

reviews_df_base = pd.read_json('reviews_df.json', lines=True)

user_items_df_base = pd.read_json('user_items_df.json', lines=True)

In [4]:
steam_games_df = steam_games_df_base

reviews_df = reviews_df_base

user_items_df = user_items_df_base

# Preprocesamiento

In [5]:
#steam games
steam_games_df = steam_games_df[steam_games_df['item_id'].notna()]
steam_games_df.reset_index(drop=True, inplace=True)
steam_games_df['item_id'] = steam_games_df['item_id'].astype(int)

In [6]:
steam_games_df

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,item_id,developer,sentiment
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,None
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,None
4,,[],Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32128,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.49,http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS""",None
32129,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",4.24,http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada,None
32130,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",1.39,http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich,None
32131,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",NaN,http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns",1 user reviews


In [7]:
#reviews 
reviews_df = reviews_df[reviews_df['item_id'].notna()]
reviews_df.reset_index(drop=True, inplace=True)
reviews_df['item_id'] = reviews_df['item_id'].astype(int)
#Si no esta en nuestra base de steam games. 
reviews_df = reviews_df[reviews_df['item_id'].isin(steam_games_df['item_id'])]
reviews_df = reviews_df.drop(columns=['funny', 'posted', 'last_edited', 'helpful', 'user_url'], axis=1)

In [8]:
reviews_df

,item_id,recommend,review,user_id
0,1250,1.0,Simple yet with great replayability. In my opi...,76561197970982479
1,22200,1.0,It's unique and worth a playthrough.,76561197970982479
4,227300,1.0,For a simple (it's actually not all that simpl...,js41637
5,239030,1.0,Very fun little game to play when your bored o...,js41637
6,248820,1.0,A suitably punishing roguelike platformer. Wi...,evcentric
...,...,...,...,...
59300,70,1.0,a must have classic from steam definitely wort...,76561198312638244
59301,362890,1.0,this game is a perfect remake of the original ...,76561198312638244
59302,273110,1.0,had so much fun plaing this and collecting res...,LydiaMorley
59303,730,1.0,:D,LydiaMorley


In [9]:
# Este cambio se hizo antes -> steam_games_df.rename(columns={'id': 'item_id'}, inplace=True)
#User Items
user_items_df = user_items_df[user_items_df['item_id'].isin(steam_games_df['item_id'])]
user_items_df['item_id'] = user_items_df['item_id'].astype(int)
user_items_df = user_items_df.drop(columns=['user_url'], axis=1)

In [10]:
user_items_df = user_items_df[user_items_df['playtime_forever'] != 0.0]
user_items_df

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6.0,0.0,76561197970982479
2,30,Day of Defeat,7.0,0.0,76561197970982479
8,300,Day of Defeat: Source,4733.0,0.0,76561197970982479
9,240,Counter-Strike: Source,1853.0,0.0,76561197970982479
10,3830,Psychonauts,333.0,0.0,76561197970982479
...,...,...,...,...,...
5169828,417860,Emily is Away,8.0,0.0,76561198323066619
5170007,304930,Unturned,677.0,677.0,76561198329548331
5170008,227940,Heroes & Generals,43.0,43.0,76561198329548331
5170011,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331


In [11]:
def sigmoid(x, scale=1):
    return 1 / (1 + np.exp(-x * scale))
user_items_df['log_ptime'] = np.log1p(user_items_df['playtime_forever'])
user_items_df['exp_ptime_2week'] = sigmoid(user_items_df['playtime_2weeks'], scale=0.3) 
max_value = 10 
user_items_df['exp_ptime_2week'] *= max_value

In [12]:
scaler_playtime = MinMaxScaler(feature_range=(0, 5))
user_items_df['log_ptime_scaled'] = scaler_playtime.fit_transform(user_items_df[['log_ptime']].values.reshape(-1, 1))

scaler_lastweek = MinMaxScaler(feature_range=(0, 5))
user_items_df['exp_ptime_2week_scaled'] = scaler_lastweek.fit_transform(user_items_df[['exp_ptime_2week']].values.reshape(-1, 1))

In [13]:
n=0.6 # Peso para el tiempo de juego en las últimas dos semanas
m=0.4 # Peso para el tiempo de juego total
user_items_df['combined_rating'] = n * user_items_df['exp_ptime_2week_scaled'] + m * user_items_df['log_ptime_scaled']

In [14]:
user_items_df = pd.merge(user_items_df, reviews_df[['user_id', 'item_id', 'recommend']], on=['user_id', 'item_id'], how='left')
#result = pd.merge(user_items_df, reviews_df, on=['user_id', 'item_id'],how='left')


In [15]:
user_items_df

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,log_ptime,exp_ptime_2week,log_ptime_scaled,exp_ptime_2week_scaled,combined_rating,recommend
0,10,Counter-Strike,6.0,0.0,76561197970982479,1.945910,5.000000,0.493976,0.000000,0.197590,NaN
1,30,Day of Defeat,7.0,0.0,76561197970982479,2.079442,5.000000,0.546629,0.000000,0.218651,NaN
2,300,Day of Defeat: Source,4733.0,0.0,76561197970982479,8.462526,5.000000,3.063538,0.000000,1.225415,NaN
3,240,Counter-Strike: Source,1853.0,0.0,76561197970982479,7.525101,5.000000,2.693903,0.000000,1.077561,NaN
4,3830,Psychonauts,333.0,0.0,76561197970982479,5.811141,5.000000,2.018072,0.000000,0.807229,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2869328,417860,Emily is Away,8.0,0.0,76561198323066619,2.197225,5.000000,0.593072,0.000000,0.237229,NaN
2869329,304930,Unturned,677.0,677.0,76561198329548331,6.519147,10.000000,2.297246,5.000000,3.918898,NaN
2869330,227940,Heroes & Generals,43.0,43.0,76561198329548331,3.784190,9.999975,1.218827,4.999975,3.487516,NaN
2869331,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,1.386294,7.109495,0.273314,2.109495,1.375023,NaN


In [16]:
#result

In [17]:
user_items_df['weighted_rating'] = user_items_df['combined_rating'] * (1 + user_items_df['recommend'].fillna(0))

In [18]:
user_item_matrix = user_items_df.pivot_table(values='weighted_rating', index='item_id', columns='user_id', fill_value=0)

In [19]:
user_item_matrix

user_id,--000--,--ace--,--ionex--,-2SV-vuLB-Kg,-404PageNotFound-,-AnimeIsMyThing-,-Azsael-,-Beave-,-Encore-,-GM-Dragon,...,zyxwvutsrqponm,zyzzchris,zzbrunozz,zzeee,zzjang2,zzonci,zzoptimuszz,zzydrax,zzyfo,zzzmidmiss
item_id,,,,,,,,,,,,,,,,,,,,,
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.317798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.173277,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.541621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527510,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
527810,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
527890,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [20]:
user_item_matrix.shape

(8523, 67894)

In [21]:
from scipy.sparse import csc_matrix

sparse_user_item_matrix = csc_matrix(user_item_matrix.values)


In [22]:
U, sigma, VT = svds(sparse_user_item_matrix, k=100)  
sigma = np.diag(sigma)

# Compute the dot product to get the approximated original matrix
predicted_ratings = np.dot(np.dot(U, sigma), VT)

# Converting the reconstructed matrix back to a DataFrame
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=user_item_matrix.index, columns=user_item_matrix.columns)

In [23]:

# Flatten the matrices to get vectors of all ratings
actual_ratings = user_item_matrix.values.flatten()
predicted_ratings = predicted_ratings_df.values.flatten()

# Only consider non-zero elements
mask = actual_ratings > 0
actual_ratings = actual_ratings[mask]
predicted_ratings = predicted_ratings[mask]

# Calculate RMSE
rmse_svd = sqrt(mean_squared_error(actual_ratings, predicted_ratings))
print("RMSE:", rmse_svd)

RMSE: 0.6507207503952525


In [24]:
# def recommend_items_svd(user_id, original_matrix, predictions_df, num_recommendations=5):
#     # Find the column for the user in the predictions matrix
#     user_col_number = original_matrix.columns.get_loc(user_id)
#     
#     # Get actual user data and apply mask to filter out items already rated (non-zero ratings)
#     user_actual_ratings = original_matrix[user_id]
#     non_zero_mask = user_actual_ratings > 0  # True for items the user has rated
#     
#     # Generate predictions for the user, filtering out already rated items
#     user_predictions = predictions_df.iloc[:, user_col_number].sort_values(ascending=False)
#     recommended_items = user_predictions[~non_zero_mask].head(num_recommendations)
#     
#     return recommended_items
def recommend_items_svd(user_id, original_matrix, predictions_df, num_recommendations=5):
    user_col_number = original_matrix.columns.get_loc(user_id)    
    sorted_user_predictions = predictions_df.iloc[:, user_col_number].sort_values(ascending=False)
    #print(sorted_user_predictions)
    user_data = original_matrix[user_id]
    #print(user_data[user_data != 0].index)
    return sorted_user_predictions.head(num_recommendations)
    # return recommendations.head(num_recommendations)

In [25]:
user_id = 'evcentric'  # Example user ID
recommendations = recommend_items_svd(user_id, user_item_matrix, predicted_ratings_df, num_recommendations=5)

recommendations_df = pd.DataFrame(recommendations).reset_index()
recommendations_df.columns = ['item_id', 'predicted_rating']
merged_df = recommendations_df.merge(steam_games_df, left_on='item_id', right_on='item_id', how='left')
merged_df

,item_id,predicted_rating,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment
0,230410,4.038160,Digital Extremes,"[Action, Free to Play]",Warframe,Warframe,http://store.steampowered.com/app/230410/Warfr...,2013-03-25,"[Free to Play, Action, Co-op, Multiplayer, Thi...",NaN,http://steamcommunity.com/app/230410/reviews/?...,"[Single-player, Multi-player, Co-op, Steam Tra...",Free to Play,False,Digital Extremes,Very Positive
1,211820,3.566726,Chucklefish,"[Action, Adventure, Casual, Indie, RPG]",Starbound,Starbound,http://store.steampowered.com/app/211820/Starb...,2016-07-22,"[Sandbox, Adventure, Survival, Indie, Crafting...",NaN,http://steamcommunity.com/app/211820/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",14.99,False,Chucklefish,Very Positive
2,49520,1.358669,"2K Games, Aspyr (Mac &amp;amp; Linux)","[Action, RPG]",Borderlands 2,Borderlands 2,http://store.steampowered.com/app/49520/Border...,2012-09-17,"[FPS, Co-op, Action, RPG, Loot, Shooter, Open ...",NaN,http://steamcommunity.com/app/49520/reviews/?b...,"[Single-player, Co-op, Steam Achievements, Ful...",19.99,False,"Gearbox Software,Aspyr (Mac &amp; Linux)",Overwhelmingly Positive
3,377160,1.341019,Bethesda Softworks,[RPG],Fallout 4,Fallout 4,http://store.steampowered.com/app/377160/Fallo...,2015-11-09,"[Open World, Post-apocalyptic, Exploration, Si...",NaN,http://steamcommunity.com/app/377160/reviews/?...,"[Single-player, Steam Achievements, Full contr...",29.99,False,Bethesda Game Studios,Mixed
4,22380,1.336249,Bethesda Softworks,"[Action, RPG]",Fallout: New Vegas,Fallout: New Vegas,http://store.steampowered.com/app/22380/Fallou...,2010-10-19,"[Open World, RPG, Post-apocalyptic, Singleplay...",NaN,http://steamcommunity.com/app/22380/reviews/?b...,"[Single-player, Steam Achievements, Partial Co...",9.99,False,Obsidian Entertainment,Overwhelmingly Positive


In [26]:
# def gen_recom_ground_truth_svd(num_recommendations=5):
#     recommendations = {}
#     ground_truth = {}
# 
#     for user_id in user_item_matrix.columns:
#         # Explicitly apply a mask to get non-zero rated items
#         user_ratings = user_item_matrix[user_id]
#         mask_non_zero = user_ratings > 0
#         ground_truth[user_id] = user_ratings[mask_non_zero].index.tolist()
# 
#         # Generate recommendations by first masking out known items
#         user_predictions = predicted_ratings_df.loc[:, user_id].sort_values(ascending=False)
#         # Filter out known items to focus only on items the user hasn't interacted with
#         recommendations[user_id] = user_predictions[~mask_non_zero].head(num_recommendations).index.tolist()
# 
#     return recommendations, ground_truth

In [27]:
from tqdm import tqdm
import concurrent.futures
# 
# def gen_recom_ground_truth_svd(numero_recom):
#     recommendations = {}
#     ground_truth = {}
# 
#     unique_user_ids = user_items_df['user_id'].unique()
#     for user_id in tqdm(unique_user_ids, desc="Processing Users"):
#         user_ra = user_items_df[user_items_df['user_id'] == user_id]
#         user_ratings = user_item_matrix[user_id]
#         ground_truth[user_id] = user_ra['item_id'].tolist()
#         #print(ground_truth[user_id])
#         #ground_truth[user_id] = user_ratings[user_ratings].index.tolist()
# 
#         # Obtener recomendaciones para el usuario usando SVD
#         recommended_items = recommend_items_svd(user_id, user_item_matrix, predicted_ratings_df, num_recommendations=numero_recom)
# 
#         # Convertir las puntuaciones predichas a solo IDs de juegos
#         recommendations[user_id] = list(recommended_items)[:numero_recom]
# 
#     return recommendations, ground_truth

In [28]:

# start_time_rec_svd = time.time()
# print(start_time_rec_svd)
# recommendations_dict_svd, ground_truth_dict_svd = gen_recom_ground_truth_svd(numero_recom=10)
# end_time_rec_svd = time.time()


In [29]:
import os

num_cores = os.cpu_count()

print("Number of cores available:", num_cores)

Number of cores available: 12


In [29]:
from pathos.multiprocessing import ProcessingPool as Pool

def process_user(user_id, numero_recom):
    user_ra = user_items_df[user_items_df['user_id'] == user_id]
    ground_truth = user_ra['item_id'].tolist()
    
    # Get recommendations for the user using SVD
    recommendations_series = recommend_items_svd(user_id, user_item_matrix, predicted_ratings_df, num_recommendations=numero_recom)
    
    # Convert Series to DataFrame, reset the index to make 'item_id' a column
    recommendations_df = recommendations_series.reset_index()
    recommendations_df.columns = ['item_id', 'predicted_rating']
    
    return user_id, recommendations_df, ground_truth


def gen_recom_ground_truth_svd(numero_recom, num_cores):
    recommendations = {}
    ground_truth = {}
    # Fetch unique user IDs
    unique_user_ids = user_items_df['user_id'].unique()
    
    # Create a Pool with the specified number of cores
    with Pool(num_cores) as pool:
        results = list(tqdm(pool.imap(lambda user_id: process_user(user_id, numero_recom), unique_user_ids),
                            total=len(unique_user_ids), desc="Processing Users"))

    # Collect results from multiprocessing
    for user_id, user_recommendations, user_ground_truth in results:
        recommendations[user_id] = user_recommendations
        ground_truth[user_id] = user_ground_truth

    return recommendations, ground_truth

if __name__ == '__main__':
    import time
    start_time_rec_svd = time.time()
    numero_recom = 10
    num_cores =10
    recommendations_df_dict_svd, ground_truth_dict_svd = gen_recom_ground_truth_svd(numero_recom, num_cores)
    end_time_rec_svd = time.time()

    print(f"Time taken: {end_time_rec_svd - start_time_rec_svd} seconds")


Processing Users: 100%|██████████| 67894/67894 [28:18<00:00, 39.97it/s] 

Time taken: 1699.109354019165 seconds


In [30]:
import pickle

# Save the recommendations to a file
with open('recommendations_dict_svd.pkl', 'wb') as f:
    pickle.dump(recommendations_df_dict_svd, f)

# Save the ground truth data to a file
with open('ground_truth_dict_svd.pkl', 'wb') as f:
    pickle.dump(ground_truth_dict_svd, f)

In [31]:
#SVD simple
#recommendations_svd = [recommendations_dict_svd[user] for user in recommendations_dict_svd]
recommendations_svd = [list(recommendations_df_dict_svd[user]['item_id']) for user in recommendations_df_dict_svd]
ground_truth_svd = [ground_truth_dict_svd[user] for user in ground_truth_dict_svd]

In [ ]:
#recommendations_svd

In [ ]:
#ground_truth_svd 


In [38]:

def precision_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    
    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / k

def recall_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    
    predicted = predicted[:k]
    relevant = len(set(predicted) & set(actual))
    return relevant / len(actual)

def apk(actual, predicted, k=10):
    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def ndcg_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    
    predicted = predicted[:k]
    idcg = sum([1.0 / np.log2(i + 2) for i in range(len(actual))])
    dcg = sum([1.0 / np.log2(i + 2) for i, p in enumerate(predicted) if p in actual])
    return dcg / idcg

def mrr_at_k(actual, predicted, k=10):
    if not actual:
        return 0.0
    
    predicted = predicted[:k]
    for i, p in enumerate(predicted):
        if p in actual:
            return 1.0 / (i + 1.0)
    return 0.0



def evaluate_recommendations(ground_truth, recommendations, k=5):
    metrics = {
        'Precision@K': np.mean([precision_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'Recall@K': np.mean([recall_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MAP@K': mapk(ground_truth, recommendations, k),
        'NDCG@K': np.mean([ndcg_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'MRR@K': np.mean([mrr_at_k(a, p, k) for a, p in zip(ground_truth, recommendations)]),
        'RMSE' : rmse_svd
    }
    return metrics

metrics_svd = evaluate_recommendations(ground_truth_svd, recommendations_svd, k=10)

In [39]:
metrics_svd

{'Precision@K': 0.7756208206910774,
 'Recall@K': 0.40702296528662807,
 'MAP@K': 0.8661968746296219,
 'NDCG@K': 0.5169538790966264,
 'MRR@K': 0.979523437795892,
 'RMSE': 0.6507207503952525}

In [40]:
metrics_df = pd.DataFrame.from_dict(metrics_svd, orient='index', columns=['SVD']).rename_axis('Metrics')
metrics_df

,SVD
Metrics,
Precision@K,0.775621
Recall@K,0.407023
MAP@K,0.866197
NDCG@K,0.516954
MRR@K,0.979523
RMSE,0.650721
